In [1]:
from torch.utils.data import DataLoader
import math
from sentence_transformers import models, losses
from sentence_transformers import SentencesDataset, LoggingHandler, SentenceTransformer, util, InputExample
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator, SequentialEvaluator
from sentence_transformers.datasets import ParallelSentencesDataset
import logging
from datetime import datetime
import sys
import os
import gzip
import csv
import numpy as np
import zipfile
import io
import tqdm
from glob import glob

In [2]:
train_folder = 'multilingual_nli'

In [4]:
teacher_model_name = 'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2'   #output/BSL_tuning_output-unsup-simcse-mBert-2021-11-12_04-49-02
student_model_name = 'microsoft/Multilingual-MiniLM-L12-H384' 

In [5]:
teacher_emb_model = models.Transformer(teacher_model_name, max_seq_length=32)
teacher_dimension = teacher_emb_model.get_word_embedding_dimension()
del teacher_emb_model
teacher_model = SentenceTransformer(teacher_model_name)

NameError: name 'max_seq_length' is not defined

In [6]:
student_word_embedding_model = models.Transformer(student_model_name, max_seq_length=32)
student_dimension = student_word_embedding_model.get_word_embedding_dimension()
student_pooling_model = models.Pooling(student_dimension)

In [7]:
if teacher_dimension != student_dimension:
    dense_model = models.Dense(in_features=student_dimension, out_features=teacher_dimension, activation_function=nn.Tanh())
    student_model = SentenceTransformer(modules=[student_word_embedding_model, student_pooling_model,dense_model])
else:
    student_model = SentenceTransformer(modules=[student_word_embedding_model, student_pooling_model])

In [52]:
train_files = []
for f in glob(f"{train_folder}/en-*.train"):
    train_files.append(f)

train_data = ParallelSentencesDataset(student_model=student_model, teacher_model=teacher_model, batch_size=64, use_embedding_cache=True)
for train_file in train_files[1:]:
    train_data.load_data(train_file)
    



KeyboardInterrupt: 

In [75]:
now = train_data.datasets

In [78]:
now[0]

[('A man is taking a swing with his axe in front of chopped wood . ',
  {'A man is taking a swing with his axe in front of chopped wood . ',
   'Bir adam doğranmış odun önünde baltasıyla sallanıyor.'}),
 ('The performance elements and standards include the following . ',
  {'Performans unsurları ve standartları aşağıdakileri içerir.',
   'The performance elements and standards include the following . '}),
 ('the lady and the kid are walking . ',
  {'bayan ve çocuk yürüyor.', 'the lady and the kid are walking . '}),
 ('There is a man cutting up meat . ',
  {'Et kesen bir adam var.', 'There is a man cutting up meat . '}),
 ('He is malevolent and makes things worse . ',
  {'He is malevolent and makes things worse . ',
   'Kötü niyetli ve işleri daha da kötüleştiriyor.'}),
 ('There is a college player being tackled in a game . ',
  {'Bir oyunda ele alınan bir kolej oyuncusu var.',
   'There is a college player being tackled in a game . '}),
 ('Three figures are dwarfed by a construction si

In [77]:
for x in now[0][:10]:
    print(x[0])
#     for s in x[1]:
#         print(s)

A man is taking a swing with his axe in front of chopped wood . 
The performance elements and standards include the following . 
the lady and the kid are walking . 
There is a man cutting up meat . 
He is malevolent and makes things worse . 
There is a college player being tackled in a game . 
Three figures are dwarfed by a construction site . 
Three woman are outdoors leaning on a fence 
The border between the military museum and its gift shop was porous . 
Two boys play in the sand 


In [10]:
train_dataloader_distill = DataLoader(train_data, shuffle=False, batch_size=64)

In [12]:
train_dataloader_distill.num_sentences

AttributeError: 'DataLoader' object has no attribute 'num_sentences'